In [ ]:
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from cond_transformer_pos_enc import Transformer
from split_dataset import VAEDataset
import matplotlib.pyplot as plt
import json
import os 
# from metrics import * 
import shutil
import bezier
from server import main, main_8bar, normalize_data_122223
import numpy as np

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [ ]:
# all_mechanisms = {'Watt2T1A1': 0, 'Type831-2': 1, 'Steph1T3': 2, 'Type823-1': 3, 'Type824-5': 4, 'Type816-0': 5, 'Type812-0': 6, 'Steph2T2A2': 7, 'Type816-2': 8, 'Type822-18': 9, 'Type814-14': 10, 'Type825-3': 11, 'Type817-9': 12, 'Type818-2': 13, 'Type823-2': 14, 'Type821-5': 15, 'Type823-7': 16, 'Type825-5': 17, 'Type816-6': 18, 'Type817-0': 19, 'Watt1T1A1': 20, 'Type816-3': 21, 'Type824-0': 22, 'Type824-8': 23, 'Type816-7': 24, 'Type814-1': 25, 'Type815-2': 26, 'Steph3T2A2': 27, 'Type821-8': 28, 'Type814-9': 29, 'Type822-9': 30, 'Type815-3': 31, 'Type814-17': 32, 'Type819-2': 33, 'Type823-11': 34, 'Type821-1': 35, 'Type818-3': 36, 'Type814-16': 37, 'Watt1T3A2': 38, 'Type823-5': 39, 'Steph2T2A1': 40, 'Type813-2': 41, 'Type824-12': 42, 'Type817-5': 43, 'Type822-12': 44, 'Type822-14': 45, 'Type822-11': 46, 'Type817-7': 47, 'Type816-9': 48, 'Type815-4': 49, 'RRRR': 50, 'Type824-4': 51, 'Type822-6': 52, 'Type814-15': 53, 'Type811-4': 54, 'Type821-6': 55, 'Type815-5': 56, 'Type816-11': 57, 'Type815-7': 58, 'Type831-0': 59, 'Type812-5': 60, 'Type814-18': 61, 'Type812-7': 62, 'Type825-4': 63, 'Type818-0': 64, 'Type824-13': 65, 'Type814-5': 66, 'Type822-2': 67, 'Type814-10': 68, 'Type815-6': 69, 'Type822-8': 70, 'Type811-2': 71, 'Type832-2': 72, 'Steph2T1A2': 73, 'Steph3T1A2': 74, 'Type822-4': 75, 'Type824-6': 76, 'Type816-4': 77, 'Type831-3': 78, 'Type817-8': 79, 'Type823-15': 80, 'Type832-3': 81, 'Type821-4': 82, 'Watt2T2A2': 83, 'Type832-4': 84, 'Type823-10': 85, 'Type825-0': 86, 'Type823-0': 87, 'Type822-0': 88, 'Type823-13': 89, 'Type814-13': 90, 'Watt1T2A1': 91, 'Type822-10': 92, 'Type824-7': 93, 'Type823-12': 94, 'Type823-8': 95, 'Type822-16': 96, 'Type815-1': 97, 'Type824-9': 98, 'Type823-6': 99, 'Type821-2': 100, 'Type822-15': 101, 'Type824-14': 102, 'Type817-3': 103, 'Type824-11': 104, 'Type819-1': 105, 'Type821-3': 106, 'Type814-7': 107, 'Type823-14': 108, 'Type825-2': 109, 'Type811-3': 110, 'Steph2T1A1': 111, 'Steph3T2A1': 112, 'Type818-1': 113, 'Type814-6': 114, 'Type812-2': 115, 'Type824-10': 116, 'Type816-5': 117, 'Type824-15': 118, 'Type811-0': 119, 'Type824-2': 120, 'Type823-3': 121, 'Type813-0': 122, 'Type821-0': 123, 'Type822-17': 124, 'Type823-4': 125, 'Watt1T1A2': 126, 'Type817-2': 127, 'Type814-0': 128, 'Type814-19': 129, 'Type811-1': 130, 'Type812-6': 131, 'Type815-0': 132, 'Type817-10': 133, 'Watt2T1A2': 134, 'Type822-3': 135, 'Steph1T2': 136, 'Type822-13': 137, 'Steph1T1': 138, 'Type822-7': 139, 'Watt1T3A1': 140, 'Type824-3': 141, 'Type815-8': 142, 'Type812-4': 143, 'Type813-1': 144, 'Type831-1': 145, 'Type817-4': 146, 'Type814-8': 147, 'Type832-0': 148, 'Type822-1': 149, 'Watt1T2A2': 150, 'Type823-9': 151, 'Type812-1': 152, 'Type819-0': 153, 'Type816-1': 154, 'Type822-19': 155, 'Type816-10': 156, 'Type815-9': 157, 'Type814-11': 158, 'Type816-8': 159, 'Type814-4': 160, 'Type814-2': 161, 'Type822-5': 162, 'Watt2T2A1': 163, 'Type814-12': 164, 'Type821-9': 165, 'Type824-1': 166, 'Type814-3': 167, 'Type817-6': 168, 'Steph3T1A1': 169, 'Type821-7': 170, 'Type817-1': 171, 'Type812-3': 172, 'Type832-1': 173, 'Type825-1': 174}
all_mechanisms = {'RRRR': 0, 'Steph1T1': 1, 'Steph1T2': 2, 'Steph1T3': 3, 'Steph2T1A1': 4, 'Steph2T1A2': 5, 'Steph2T2A1': 6, 'Steph2T2A2': 7, 'Steph3T1A1': 8, 'Steph3T1A2': 9, 'Steph3T2A1': 10, 'Steph3T2A2': 11, 'Type811-0': 12, 'Type811-1': 13, 'Type811-2': 14, 'Type811-3': 15, 'Type811-4': 16, 'Type812-0': 17, 'Type812-1': 18, 'Type812-2': 19, 'Type812-3': 20, 'Type812-4': 21, 'Type812-5': 22, 'Type812-6': 23, 'Type812-7': 24, 'Type813-0': 25, 'Type813-1': 26, 'Type813-2': 27, 'Type814-0': 28, 'Type814-1': 29, 'Type814-10': 30, 'Type814-11': 31, 'Type814-12': 32, 'Type814-13': 33, 'Type814-14': 34, 'Type814-15': 35, 'Type814-16': 36, 'Type814-17': 37, 'Type814-18': 38, 'Type814-19': 39, 'Type814-2': 40, 'Type814-3': 41, 'Type814-4': 42, 'Type814-5': 43, 'Type814-6': 44, 'Type814-7': 45, 'Type814-8': 46, 'Type814-9': 47, 'Type815-0': 48, 'Type815-1': 49, 'Type815-2': 50, 'Type815-3': 51, 'Type815-4': 52, 'Type815-5': 53, 'Type815-6': 54, 'Type815-7': 55, 'Type815-8': 56, 'Type815-9': 57, 'Type816-0': 58, 'Type816-1': 59, 'Type816-10': 60, 'Type816-11': 61, 'Type816-2': 62, 'Type816-3': 63, 'Type816-4': 64, 'Type816-5': 65, 'Type816-6': 66, 'Type816-7': 67, 'Type816-8': 68, 'Type816-9': 69, 'Type817-0': 70, 'Type817-1': 71, 'Type817-10': 72, 'Type817-2': 73, 'Type817-3': 74, 'Type817-4': 75, 'Type817-5': 76, 'Type817-6': 77, 'Type817-7': 78, 'Type817-8': 79, 'Type817-9': 80, 'Type818-0': 81, 'Type818-1': 82, 'Type818-2': 83, 'Type818-3': 84, 'Type819-0': 85, 'Type819-1': 86, 'Type819-2': 87, 'Type821-0': 88, 'Type821-1': 89, 'Type821-2': 90, 'Type821-3': 91, 'Type821-4': 92, 'Type821-5': 93, 'Type821-6': 94, 'Type821-7': 95, 'Type821-8': 96, 'Type821-9': 97, 'Type822-0': 98, 'Type822-1': 99, 'Type822-10': 100, 'Type822-11': 101, 'Type822-12': 102, 'Type822-13': 103, 'Type822-14': 104, 'Type822-15': 105, 'Type822-16': 106, 'Type822-17': 107, 'Type822-18': 108, 'Type822-19': 109, 'Type822-2': 110, 'Type822-3': 111, 'Type822-4': 112, 'Type822-5': 113, 'Type822-6': 114, 'Type822-7': 115, 'Type822-8': 116, 'Type822-9': 117, 'Type823-0': 118, 'Type823-1': 119, 'Type823-10': 120, 'Type823-11': 121, 'Type823-12': 122, 'Type823-13': 123, 'Type823-14': 124, 'Type823-15': 125, 'Type823-2': 126, 'Type823-3': 127, 'Type823-4': 128, 'Type823-5': 129, 'Type823-6': 130, 'Type823-7': 131, 'Type823-8': 132, 'Type823-9': 133, 'Type824-0': 134, 'Type824-1': 135, 'Type824-10': 136, 'Type824-11': 137, 'Type824-12': 138, 'Type824-13': 139, 'Type824-14': 140, 'Type824-15': 141, 'Type824-2': 142, 'Type824-3': 143, 'Type824-4': 144, 'Type824-5': 145, 'Type824-6': 146, 'Type824-7': 147, 'Type824-8': 148, 'Type824-9': 149, 'Type825-0': 150, 'Type825-1': 151, 'Type825-2': 152, 'Type825-3': 153, 'Type825-4': 154, 'Type825-5': 155, 'Type831-0': 156, 'Type831-1': 157, 'Type831-2': 158, 'Type831-3': 159, 'Type832-0': 160, 'Type832-1': 161, 'Type832-2': 162, 'Type832-3': 163, 'Type832-4': 164, 'Watt1T1A1': 165, 'Watt1T1A2': 166, 'Watt1T2A1': 167, 'Watt1T2A2': 168, 'Watt1T3A1': 169, 'Watt1T3A2': 170, 'Watt2T1A1': 171, 'Watt2T1A2': 172, 'Watt2T2A1': 173, 'Watt2T2A2': 174}
reversed_mechanisms = {v: k for k, v in all_mechanisms.items()}
max_mech_size = 14
num_classes = len(all_mechanisms)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Transformer(output_size=2, tgt_seq_len=max_mech_size)
checkpoint = torch.load("weights/ROTATION_CODITIONAL_N_12.ckpt")
model.load_state_dict(checkpoint['state_dict'], strict=False)
model.eval()

In [ ]:
dataset = VAEDataset(num_classes=num_classes, max_mech_size=max_mech_size)
data_loader = DataLoader(dataset, shuffle=True, batch_size=1)

In [ ]:
def greedy_decode_conditional(model, source, mech_type, max_len, eos_token=torch.tensor([1.0, 1.0])):
    model, source = model.to(device), source.to(device)
    mech_type = mech_type.to(device)

    # Encode the source once with conditioning on mech_type
    encoder_output = model.encode(source, None, mech_type)

    # Initial inputs for the decoders
    decoder_input_first = torch.zeros(1, 1, 2).to(device)  # Start token for decoder 1
    decoder_input_second = torch.zeros(1, 1, 2).to(device)  # Start token for decoder 2

    # Decoded outputs for both decoders
    decoded_first = []
    decoded_second = []

    # First decoding loop
    while decoder_input_first.size(1) < max_len // 2:
        # Build causal mask for decoder 1
        decoder_mask_first = causal_mask(decoder_input_first.size(1)).type_as(decoder_input_first).to(device)

        # Decode using the first decoder
        decoder_output_first = model.decode(
            model.decoder_first,
            encoder_output,
            None,
            decoder_input_first,
            decoder_mask_first
        )
        proj_output_first = model.projection_first(model.projection_norm_first(decoder_output_first))

        # Get the next predicted token
        next_token_first = proj_output_first[:, -1].unsqueeze(0)

        # Append to the decoded sequence
        decoded_first.append(next_token_first.squeeze(0).detach().cpu())

        # Check for the EOS token
        if torch.allclose(next_token_first.squeeze(), eos_token.to(device), atol=1e-1):
            break

        # Append the next token to the input sequence for further decoding
        decoder_input_first = torch.cat([decoder_input_first, next_token_first], dim=1)

    # Second decoding loop
    while decoder_input_second.size(1) < max_len // 2:
        # Build causal mask for decoder 2
        decoder_mask_second = causal_mask(decoder_input_second.size(1)).type_as(decoder_input_second).to(device)

        # Decode using the second decoder
        decoder_output_second = model.decode(
            model.decoder_second,
            encoder_output,
            None,
            decoder_input_second,
            decoder_mask_second
        )
        proj_output_second = model.projection_second(model.projection_norm_second(decoder_output_second))

        # Get the next predicted token
        next_token_second = proj_output_second[:, -1].unsqueeze(0)

        # Append to the decoded sequence
        decoded_second.append(next_token_second.squeeze(0).detach().cpu())

        # Check for the EOS token
        if torch.allclose(next_token_second.squeeze(), eos_token.to(device), atol=1e-1):
            break

        # Append the next token to the input sequence for further decoding
        decoder_input_second = torch.cat([decoder_input_second, next_token_second], dim=1)

    return decoder_input_first.squeeze(0), decoder_input_second.squeeze(0)


def causal_mask(size):
    mask = torch.triu(torch.ones((1, size, size)), diagonal=1).type(torch.int)
    return mask == 0


In [ ]:
# import os
# import json
# import shutil
# import numpy as np
# import torch
# import matplotlib.pyplot as plt

# # Remove existing results directory and recreate it
# if os.path.exists('results'):
#     shutil.rmtree('results')
# os.makedirs('results', exist_ok=True)

# # Configuration
# max_mech_size = 14
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # Load BSI data
# with open('BSIdict_468.json', 'r') as file:
#     bsi_data = json.load(file)

# def filter_invalid_joints(joints, invalid_values):
#     mask = torch.ones(joints.size(0), dtype=torch.bool, device=joints.device)
#     for invalid_value in invalid_values:
#         mask &= ~(joints == invalid_value).all(dim=1)
#     return joints[mask].detach().cpu().numpy()

# # Iterate over batches of input data
# for batch_num, batch in enumerate(data_loader):
#     if batch_num == 100:  # Limit to 10 input coupler curves
#         break

#     # Extract input data
#     img_tensor = batch["encoder_input"].to(device)

#     # Combine `label_first` and `label_second` into a single tensor
#     truth_joints = torch.cat([
#         batch["label_first"].to(device).view(-1, 2),
#         batch["label_second"].to(device).view(-1, 2)
#     ], dim=0)

#     # Filter out invalid joints
#     invalid_values = [torch.tensor([0.5, 0.5], device=device), torch.tensor([1.0, 1.0], device=device)]
#     truth_joints = filter_invalid_joints(truth_joints, invalid_values)

#     # Generate ground truth coupler curve (same for all mechanism types)
#     mech_type_idx = torch.argmax(batch["mech_type"], dim=1).item()
#     mech_type = reversed_mechanisms.get(mech_type_idx)

#     if mech_type.startswith('T'):
#         original_temp = main_8bar(truth_joints, mech_type)
#     else:
#         original_temp = main(truth_joints, mech_type)

#     # Skip if original data generation fails
#     if original_temp is None:
#         continue

#     original_data = np.asarray(original_temp[0]['poses'])
#     coupler_idx = bsi_data[mech_type]['c'].index(1)
#     original_data = original_data[:, coupler_idx, :]
#     original_data, _, _ = normalize_data_122223(original_data, scaling=1.0)
#     original_x = original_data[:, 0]
#     original_y = original_data[:, 1]

#     # Create a folder for this input coupler curve
#     curve_results_dir = os.path.join('results', str(batch_num + 1))
#     os.makedirs(curve_results_dir, exist_ok=True)

#     # Iterate over all possible mechanism types
#     for mech_idx, mech_name in reversed_mechanisms.items():
#         # Create one-hot vector for the current mechanism type
#         mech_type_one_hot = torch.zeros(len(reversed_mechanisms), dtype=torch.float32).to(device)
#         mech_type_one_hot[mech_idx] = 1.0
#         mech_type_one_hot = mech_type_one_hot.unsqueeze(0)  # Now the size is [1, 175]
        
#         coupler_idx = bsi_data[mech_name]['c'].index(1)
    
#         # Predict joints for the current mechanism type
#         pred_joints_first, pred_joints_second = greedy_decode_conditional(model, img_tensor, mech_type_one_hot, max_mech_size)
#         pred_joints_first = pred_joints_first[1:]  # Exclude start tokens
#         pred_joints_second = pred_joints_second[1:]
#         pred_joints = torch.cat([pred_joints_first, pred_joints_second], dim=0)

#         # Filter out invalid joints
#         pred_joints = filter_invalid_joints(pred_joints, invalid_values)

#         # Generate coupler curve for the predicted mechanism
#         if mech_name.startswith('T'):
#             predicted_temp = main_8bar(pred_joints, mech_name)
#         else:
#             predicted_temp = main(pred_joints, mech_name)

#         # Skip if prediction generation fails
#         if predicted_temp is None:
#             continue

#         predicted_data = np.asarray(predicted_temp[0]['poses'])
#         predicted_data = predicted_data[:, coupler_idx, :]
        
#         if len(predicted_data) < 20:
#             continue
        
#         predicted_data, _, _ = normalize_data_122223(predicted_data, scaling=1.0)

#         pred_x = predicted_data[:, 0]
#         pred_y = predicted_data[:, 1]

#         # Plot results
#         fig, ax = plt.subplots()

#         # Plot ground truth joints
#         ax.scatter(*zip(*truth_joints), color='red', label='Truth Joints')
#         ax.plot(original_x, original_y, 'r', label='Original Curve')

#         # Plot predicted joints and coupler curve
#         ax.scatter(*zip(*pred_joints), color='blue', label=f'Predicted Joints ({mech_name})')
#         ax.plot(pred_x, pred_y, 'b', label=f'Predicted Curve ({mech_name})')

#         ax.set_title(f"Input Coupler Curve {batch_num + 1} | Mechanism: {mech_name}")
#         ax.set_xlabel("X")
#         ax.set_ylabel("Y")
#         # ax.legend()

#         # Save plot to the mechanism folder
#         plt.savefig(os.path.join(curve_results_dir, f"{mech_name}.jpg"))
#         plt.close(fig)

# print("All predictions completed and saved in 'results' folder.")


In [ ]:
# Remove the results directory if it exists
if os.path.exists('results'):
    shutil.rmtree('results')

# Create device object
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the device
model = model.to(device)

# Set constants
max_mech_size = 14

# Ensure the results directory exists
os.makedirs('results', exist_ok=True)

# Function to filter invalid joints
def filter_invalid_joints(joints, invalid_values, device=None):
    """
    Filters out invalid joints from the input tensor.

    Args:
        joints (torch.Tensor): Input tensor of joints.
        invalid_values (list of torch.Tensor): List of invalid values to filter out.
        device: Torch device (optional).

    Returns:
        torch.Tensor: Filtered joints.
    """
    mask = torch.ones(joints.size(0), dtype=torch.bool, device=device)
    for value in invalid_values:
        mask &= ~(joints == value.to(joints.device)).all(dim=1)
    return joints[mask].detach().cpu()

# Iterate over the data loader
for batch_num, batch in enumerate(data_loader):
    if batch_num == 20:  # Limit the number of batches processed
        break

    # Extract the inputs and ground truth labels
    img_tensor = batch["encoder_input"].to(device)
    mech_type = batch["mech_type"].to(device)  # Mechanism type as an additional conditional input
    print(mech_type.size())
    mech_first = batch["label_first"].to(device)
    mech_second = batch["label_second"].to(device)

    # Combine ground truth joints from both labels
    truth_joints_first = mech_first.view(-1, 2)
    truth_joints_second = mech_second.view(-1, 2)
    truth_joints = torch.cat([truth_joints_first[:-1], truth_joints_second[:-1]], dim=0)

    # Get predictions from the model using the conditional greedy decode
    pred_joints_first, pred_joints_second = greedy_decode_conditional(
        model, img_tensor, mech_type, max_mech_size
        )

    # Exclude start tokens from predictions
    pred_joints_first = pred_joints_first[1:].detach().cpu()
    pred_joints_second = pred_joints_second[1:].detach().cpu()
    pred_joints = torch.cat([pred_joints_first, pred_joints_second], dim=0)

    # Remove invalid joints from ground truth and predictions
    invalid_values = [torch.tensor([0.5, 0.5]), torch.tensor([1.0, 1.0]), torch.tensor([0.0, 0.0])]
    truth_joints = filter_invalid_joints(truth_joints, invalid_values, device=device).numpy()
    pred_joints = filter_invalid_joints(pred_joints, invalid_values).numpy()

    # # Print the processed joints
    # print(f"Batch {batch_num}")
    # print("Truth Joints:\n", truth_joints)
    # print("Predicted First Joints:\n", pred_joints_first)
    # print("Predicted Second Joints:\n", pred_joints_second)

    # Plot the results
    fig, ax = plt.subplots()
    # Plot truth joints
    for i, (x, y) in enumerate(truth_joints):
        ax.scatter(x, y, color='red', label='Truth' if i == 0 else "")
        ax.text(x, y, f'{i + 1}', color='red')

    # Plot predicted joints
    for i, (x, y) in enumerate(pred_joints):
        ax.scatter(x, y, color='blue', label='Prediction' if i == 0 else "")
        ax.text(x, y, f'{i + 1}', color='blue')

    # Add labels and legend
    ax.set_title(f"Truth vs Prediction (Mechanism Type: {reversed_mechanisms[mech_type.argmax().item()]})")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.legend(loc='best')

    # Save the plot
    plt.savefig(f'results/batch_{batch_num}.jpg')
    plt.show()
